In [1]:
import pandas as pd
import geopandas
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)
import random
from scipy import stats
from scipy import integrate
import glob
import seaborn as sns
# %matplotlib notebook
%matplotlib inline
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

from fitter import Fitter, get_common_distributions, get_distributions

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# Procedure discarded since it is too old!

## Data based on Intensive Livestock Farming: Does Farm Size Matter?

Prepared by
Martin S. Beaulieu, MSc, Analyst
Analysis and Development Section
martin.beaulieu@statcan.ca

https://www150.statcan.gc.ca/n1/en/pub/21-601-m/21-601-m2001048-eng.pdf?st=MAVG2s8P

In [10]:
# # These are partial dicts beecause the total is referred over total Canada
# # The shares are of Animal Units
# DairyDistriution_Partialdict = {'1-99':0.14,
#                    '100-199':0.13,
#                    '200-299':0.02,
#                    '300-399':0.01,
#                    '>400':0.01,}

# BeefDistriution_Partialdict = {'1-99':0.13,
#                    '100-199':0.06,
#                    '200-299':0.02,
#                    '300-399':0.01,
#                    '>400':0.04,}

# SwineDistriution_Partialdict = {'1-99':0.06,
#                    '100-199':0.06,
#                    '200-299':0.02,
#                    '300-399':0.01,
#                    '>400':0.02,}

# PoultryDistriution_Partialdict = {'1-99':0.01,
#                    '100-199':0.03,
#                    '200-299':0.02,
#                    '300-399':0.01,
#                    '>400':0.03,}

# OtherDistriution_Partialdict = {'1-99':0.08,
#                    '100-199':0.04,
#                    '200-299':0.01,
#                    '300-399':0.00,
#                    '>400':0.01,}

In [13]:
# sum(OtherDistriution_Partialdict.values())

0.14

In [57]:
# LivestockTypes_dict = {'Dairy':DairyDistriution_Partialdict,
#                       'Beef':BeefDistriution_Partialdict,
#                       'Swine':SwineDistriution_Partialdict,
#                       'Poultry':PoultryDistriution_Partialdict,
#                       'Other':OtherDistriution_Partialdict}

# LivestockDistriution_df = pd.DataFrame()

# for LivestockType, Livestock_dict in zip(LivestockTypes_dict.keys(), LivestockTypes_dict.values()):
#     # name = i.rsplit('Distriution')[0]
#     Correctedd_dict = {}
#     TotalLivestockShare = sum(Livestock_dict.values())
#     for key, value in zip(Livestock_dict.keys(), Livestock_dict.values()):
#         Correctedd_dict[key] = value/TotalLivestockShare
#     Aux_df  = pd.DataFrame.from_dict(Correctedd_dict, orient='index', columns=[LivestockType + ' (% AUs)'])
#     LivestockDistriution_df = pd.concat([LivestockDistriution_df, Aux_df], axis =1)
    
    
# LivestockSizes_dict = {'1-99':50,
#                       '100-199':150,
#                       '200-299':250,
#                       '300-399':350,
#                       '>400':600}

# Aux_df  = pd.DataFrame.from_dict(LivestockSizes_dict, orient='index', columns=['Average size (AUs)'])
# LivestockDistriution_df = pd.concat([LivestockDistriution_df, Aux_df], axis =1)

In [58]:
# LivestockDistriution_df

,Dairy (% AUs),Beef (% AUs),Swine (% AUs),Poultry (% AUs),Other (% AUs),Average size (AUs)
1-99,0.451613,0.500000,0.352941,0.1,0.571429,50
100-199,0.419355,0.230769,0.352941,0.3,0.285714,150
200-299,0.064516,0.076923,0.117647,0.2,0.071429,250
300-399,0.032258,0.038462,0.058824,0.1,0.000000,350
>400,0.032258,0.153846,0.117647,0.3,0.071429,600


In [104]:
# LivestockAUs_dict = {'Dairy':1376984*0.59, #This fractions are based on Intensive Livestock Farming: Does Farm Size Matter? Prepared byMartin S. Beaulieu, MSc, Analyst 2001
#                       'Beef':1376984*(1-0.59),
#                       'Swine':506768,
#                       'Poultry':148508,
#                       'Other':52227+83498}

# LivestockP_dict = {'Dairy':7867.8*1000, #P Flows report
#                       'Beef':5405.9*1000,
#                       'Swine':8569.8*1000,
#                       'Poultry':6456.8*1000,
#                       'Other':(605.8+1677.1)*1000}

In [67]:
# for LivestockType, Livestock_dict in zip(LivestockTypes_dict.keys(), LivestockTypes_dict.values()):
#     LivestockDistriution_df[LivestockType + ' AUs distribution (AUs)'] = (LivestockDistriution_df[LivestockType + ' (% AUs)']*
#                                                                          LivestockAUs_dict[LivestockType])
    
#     LivestockDistriution_df[LivestockType + ' CAFOs (CAFOs)'] = (LivestockDistriution_df[LivestockType + ' AUs distribution (AUs)']/
#                                                                 LivestockDistriution_df['Average size (AUs)'])
                                                                          
# LivestockDistriution_df

,Dairy (% AUs),Beef (% AUs),Swine (% AUs),Poultry (% AUs),Other (% AUs),Average size (AUs),Dairy AUs distribution (AUs),Beef AUs distribution (AUs),Swine AUs distribution (AUs),Poultry AUs distribution (AUs),Other AUs distribution (AUs),Dairy CAFOs (CAFOs),Beef CAFOs (CAFOs),Swine CAFOs (CAFOs),Poultry CAFOs (CAFOs),Other CAFOs (CAFOs)
1-99,0.451613,0.500000,0.352941,0.1,0.571429,50,366899.607742,282281.720000,178859.294118,14850.8,77557.142857,7337.992155,5645.634400,3577.185882,297.016000,1551.142857
100-199,0.419355,0.230769,0.352941,0.3,0.285714,150,340692.492903,130283.870769,178859.294118,44552.4,38778.571429,2271.283286,868.559138,1192.395294,297.016000,258.523810
200-299,0.064516,0.076923,0.117647,0.2,0.071429,250,52414.229677,43427.956923,59619.764706,29701.6,9694.642857,209.656919,173.711828,238.479059,118.806400,38.778571
300-399,0.032258,0.038462,0.058824,0.1,0.000000,350,26207.114839,21713.978462,29809.882353,14850.8,0.000000,74.877471,62.039938,85.171092,42.430857,0.000000
>400,0.032258,0.153846,0.117647,0.3,0.071429,600,26207.114839,86855.913846,59619.764706,44552.4,9694.642857,43.678525,144.759856,99.366275,74.254000,16.157738


In [90]:
# LivestockDistriution_df.sum(axis=0)

Dairy (% AUs)                          1.000000
Beef (% AUs)                           1.000000
Swine (% AUs)                          1.000000
Poultry (% AUs)                        1.000000
Other (% AUs)                          1.000000
Average size (AUs)                  1400.000000
Dairy AUs distribution (AUs)      812420.560000
Beef AUs distribution (AUs)       564563.440000
Swine AUs distribution (AUs)      506768.000000
Poultry AUs distribution (AUs)    148508.000000
Other AUs distribution (AUs)      135725.000000
Dairy CAFOs (CAFOs)                 9937.488355
Beef CAFOs (CAFOs)                  6894.705161
Swine CAFOs (CAFOs)                 5192.597602
Poultry CAFOs (CAFOs)                829.523257
Other CAFOs (CAFOs)                 1864.602976
dtype: float64

In [114]:
# Dairy_CAFOs_df = pd.DataFrame()

# for index, row in LivestockDistriution_df.iterrows():

#     Cattle_CAFOsAUs = ([LivestockDistriution_df.loc[index]['Average size (AUs)'].item()]*
#                       int(np.round(LivestockDistriution_df.loc[index]['Dairy CAFOs (CAFOs)'].item())))
    
#     Dairy_CAFOs_df = Dairy_CAFOs_df.append(Cattle_CAFOsAUs)
    
# Dairy_CAFOs_df = Dairy_CAFOs_df.reset_index(drop=True)
# Dairy_CAFOs_df = Dairy_CAFOs_df.rename(columns={0:'AUs'})
# Dairy_CAFOs_df['kgPin/year'] = (Dairy_CAFOs_df['AUs']*(LivestockP_dict['Dairy']/LivestockAUs_dict['Dairy']))


# Beef_CAFOs_df = pd.DataFrame()
    
# for index, row in LivestockDistriution_df.iterrows():

#     Cattle_CAFOsAUs = ([LivestockDistriution_df.loc[index]['Average size (AUs)'].item()]*
#                       int(np.round(LivestockDistriution_df.loc[index]['Beef CAFOs (CAFOs)'].item())))
    
#     Beef_CAFOs_df = Beef_CAFOs_df.append(Cattle_CAFOsAUs)
    
# Beef_CAFOs_df = Beef_CAFOs_df.reset_index(drop=True)
# Beef_CAFOs_df = Beef_CAFOs_df.rename(columns={0:'AUs'})
# Beef_CAFOs_df['kgPin/year'] = (Beef_CAFOs_df['AUs']*(LivestockP_dict['Beef']/LivestockAUs_dict['Beef']))

# Cattle_CAFOs = pd.concat([Dairy_CAFOs_df, Beef_CAFOs_df], ignore_index=True)

# Cattle_CAFOs.to_csv('OntarioData/CAFOs_Cattle_df.csv')

# Cattle_CAFOs

,AUs,kgPin/year
0,50.0,484.219651
1,50.0,484.219651
2,50.0,484.219651
3,50.0,484.219651
4,50.0,484.219651
...,...,...
16829,600.0,5745.217933
16830,600.0,5745.217933
16831,600.0,5745.217933
16832,600.0,5745.217933


In [115]:
# Swine_CAFOs_df = pd.DataFrame()

# for index, row in LivestockDistriution_df.iterrows():

#     Swine_CAFOsAUs = ([LivestockDistriution_df.loc[index]['Average size (AUs)'].item()]*
#                       int(np.round(LivestockDistriution_df.loc[index]['Swine CAFOs (CAFOs)'].item())))
    
#     Swine_CAFOs_df = Swine_CAFOs_df.append(Swine_CAFOsAUs)
    
# Swine_CAFOs_df = Swine_CAFOs_df.reset_index(drop=True)
# Swine_CAFOs_df = Swine_CAFOs_df.rename(columns={0:'AUs'})
# Swine_CAFOs_df['kgPin/year'] = (Swine_CAFOs_df['AUs']*(LivestockP_dict['Swine']/LivestockAUs_dict['Swine']))



# Swine_CAFOs_df.to_csv('OntarioData/CAFOs_Swine_df.csv')

# Swine_CAFOs_df

,AUs,kgPin/year
0,50.0,845.534840
1,50.0,845.534840
2,50.0,845.534840
3,50.0,845.534840
4,50.0,845.534840
...,...,...
5186,600.0,10146.418085
5187,600.0,10146.418085
5188,600.0,10146.418085
5189,600.0,10146.418085


In [117]:
# Swine_CAFOs_df.mean(axis=0)

AUs             97.534194
kgPin/year    1649.371180
dtype: float64

# Work with average values from statistics Canada

In [128]:
LivestockAUs_dict = {'Cattle':1376984, #P Flows report
                      'Swine':506768,
                      'Poultry':148508,
                      'Other':52227+83498}

LivestockP_dict = {'Cattle':(7867.8+5405.9)*1000, #P Flows report
                      'Swine':8569.8*1000,
                      'Poultry':6456.8*1000,
                      'Other':(605.8+1677.1)*1000}

LivestockAvgCAFOSize_dict = {'Cattle':98/1, #In Animal UnitsFrom here: https://www150.statcan.gc.ca/n1/daily-quotidien/220228/dq220228d-cansim-eng.htm
                      'Swine':1427/((sum([5112,313698])/3534104)*2.67+(sum([503771, 882239, 1829284])/3534104)*9.09), #Average AU equivalence based on Archived - Pig inventory on farms, Census of Agriculture, 2011 and 2016, inactive https://www150.statcan.gc.ca/t1/tbl1/en/tv.action?pid=3210042601&pickMembers%5B0%5D=1.1063&cubeTimeFrame.startYear=2011&cubeTimeFrame.endYear=2016&referencePeriods=20110101%2C20160101
                      'Poultry':(50759994/8246)/((sum([5328265, 2293476, 33818351])/50759994)*455 + 9319902/50759994*250), #based on Archived - Poultry inventories on farms, Census of Agriculture, 2011 and 2016, inactive  https://www150.statcan.gc.ca/t1/tbl1/en/tv.action?pid=3210042801&pickMembers%5B0%5D=1.1063&cubeTimeFrame.startYear=2011&cubeTimeFrame.endYear=2016&referencePeriods=20110101%2C20160101
                      'Other':110/7} #In Animal Units Based on sheeps From here: https://www150.statcan.gc.ca/n1/daily-quotidien/220228/dq220228d-cansim-eng.htm

In [124]:
LivestockAvgCAFOSize_dict

{'Dairy': 98.0,
 'Swine': 167.66823932623336,
 'Poultry': 14.74914582420324,
 'Other': 15.714285714285714}

In [130]:
for LivestockType, AUs in zip(LivestockAUs_dict.keys(), LivestockAUs_dict.values()):
    
    CAFOs_df = pd.DataFrame()
    
    NumberCAFOs = AUs/LivestockAvgCAFOSize_dict[LivestockType]
    
    CAFOsAUs = ([LivestockAvgCAFOSize_dict[LivestockType]]*
                      int(np.round(NumberCAFOs)))
    
    CAFOs_df = CAFOs_df.append(CAFOsAUs)
    
    CAFOs_df = CAFOs_df.reset_index(drop=True)
    CAFOs_df = CAFOs_df.rename(columns={0:'AUs'})
    CAFOs_df['kgPin/year'] = (CAFOs_df['AUs']*(LivestockP_dict[LivestockType]/LivestockAUs_dict[LivestockType]))
    
    CAFOs_df.to_csv('OntarioData/CAFOs_' + LivestockType + '_df.csv')



CAFOs_df

,AUs,kgPin/year
0,15.714286,264.314923
1,15.714286,264.314923
2,15.714286,264.314923
3,15.714286,264.314923
4,15.714286,264.314923
...,...,...
8632,15.714286,264.314923
8633,15.714286,264.314923
8634,15.714286,264.314923
8635,15.714286,264.314923
